In [3]:
!pip install "numpy<2.0"

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 20.6 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: numpy
    Found existing installation: numpy 2.1.3
    Uninstalling numpy-2.1.3:
      Successfully uninstalled numpy-2.1.3

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [20]:
!pip uninstall numpy opencv-python -y
!pip cache purge

# Reinstall compatible versions
!pip install numpy==1.24.3 opencv-python==4.8.1.78


Found existing installation: numpy 1.24.3
Uninstalling numpy-1.24.3:
  Successfully uninstalled numpy-1.24.3
Found existing installation: opencv-python 4.11.0.86
Uninstalling opencv-python-4.11.0.86:
  Successfully uninstalled opencv-python-4.11.0.86
ERROR: pip cache commands can not function since cache is disabled.
Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 42.7 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 MB 72.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [opencv-python]0m [opencv-python]
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.19.0 requires numpy<2.2.0,>=1.26.0, but you have numpy 1.24.3 which is incompatible.


In [7]:
!pip install face-alignment opencv-python 
!pip install torch torchvision torchaudio 
!pip install scikit-learn
!pip install gdown
!pip install gradio
!pip install pyttsx3
!pip install tensorflow
!pip install numpy
# !pip install scikit-learn 
# !pip install matplotlib 
# !pip install seaborn

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip

# Installing Important Libraries

1. Importing libraries

In [3]:
import gdown
import zipfile
import os

# Clean up
if os.path.exists('data.zip'):
    os.remove('data.zip')

# Download
url = 'https://drive.google.com/file/d/1PTxVNZGC1fUcXVSIaNAnx8e4HiM7p6L9/view?usp=sharing' #my own data
#url = 'https://drive.google.com/uc?id=1YlvpDLix3S-U8fd-gqRwPcWXAXm8JwjL' #online data english
gdown.download(url, 'data.zip', quiet=False, fuzzy=True)

# Extract
with zipfile.ZipFile('data.zip', 'r') as zip_ref:
    zip_ref.extractall()

print("✅ Done!")

Downloading...
From (original): https://drive.google.com/uc?id=1PTxVNZGC1fUcXVSIaNAnx8e4HiM7p6L9
From (redirected): https://drive.google.com/uc?id=1PTxVNZGC1fUcXVSIaNAnx8e4HiM7p6L9&confirm=t&uuid=84023e5a-8922-4773-888d-c06dd5c1c0ff
To: /home/poras9868/data.zip
100%|██████████████████████████████████████████████████████████████████████| 54.7M/54.7M [00:00<00:00, 57.0MB/s]


✅ Done!


In [11]:
import os
import numpy as np
import cv2
import pickle
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Dense, LSTM, Lambda, TimeDistributed, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow as tf
from typing import List, Tuple, Optional
import torch
import face_alignment
from collections import deque
import pickle

# Check if CUDA is available
if torch.cuda.is_available():
    device = 'cuda'
    print(f"Using GPU: {torch.cuda.get_device_name()}")
else:
    device = 'cpu'
    print("Using CPU")

# Initialize face alignment model globally
fa = face_alignment.FaceAlignment(face_alignment.LandmarksType.TWO_D, device=device, flip_input=False)

# Configuration for tracking method
TRACKING_METHOD = "periodic"  # Options: "periodic", "motion", "keyframe", "lightweight", "smart"
TRACKING_CONFIG = {
    "periodic": {"recalibrate_every": 30},
    "motion": {"motion_threshold": 50000},
    "keyframe": {"keyframe_interval": 60},
    "lightweight": {"max_frames_without_detection": 90},
    "smart": {"recalibrate_every": 45}
}

Using GPU: Tesla T4


# 
1. PreProcessing

In [13]:
#Video Processing Functions
def measurements(frame) -> Tuple[int, int, int, int]:
    """Extract crop measurements from a single frame"""
    preds = fa.get_landmarks(frame)
    if not preds:
        return (190, 236, 80, 220)

    landmarks = preds[0]

    # Key landmarks
    nose_base = landmarks[33]
    chin_bottom = landmarks[8]
    lip_center = landmarks[66]
    left_lip = landmarks[48]
    right_lip = landmarks[54]
    left_cheek = landmarks[3]
    right_cheek = landmarks[13]

    # Top: just below nose
    crop_y1 = int(nose_base[1])

    # Bottom: halfway between lip center and chin
    crop_y2 = int((chin_bottom[1] + lip_center[1]) / 2)

    # Compute cheek-to-lip distances
    left_dist = np.linalg.norm(left_lip - left_cheek)
    right_dist = np.linalg.norm(right_lip - right_cheek)
    min_padding = int(min(left_dist, right_dist))

    # Apply symmetric padding from cheek landmarks
    crop_x1 = int(left_cheek[0] - min_padding)
    crop_x2 = int(right_cheek[0] + min_padding)

    # Clamp to frame boundaries
    crop_y1 = max(0, crop_y1)
    crop_y2 = min(frame.shape[0], crop_y2)
    crop_x1 = max(0, crop_x1)
    crop_x2 = min(frame.shape[1], crop_x2)

    return (crop_y1, crop_y2, crop_x1, crop_x2)

def resize_with_horizontal_padding(image, target_width: int, target_height: int = None):
    """
    Resize image to target width, and optionally target height,
    adding horizontal padding only if needed.
    """
    current_height = tf.shape(image)[0]
    current_width = tf.shape(image)[1]

    if target_height is not None:
        # Resize to exact target dimensions
        if current_width < target_width:
            # First resize to target height, then pad width
            resized = tf.image.resize(image, [target_height, current_width])
            pad_width = target_width - current_width
            paddings = [
                [0, 0],  # No vertical padding
                [pad_width // 2, pad_width - pad_width // 2],  # Horizontal padding
                [0, 0]   # No channel padding
            ]
            return tf.pad(resized, paddings, mode='CONSTANT', constant_values=0)
        else:
            # Just resize to target dimensions
            return tf.image.resize(image, [target_height, target_width])
    else:
        # Keep original height, only adjust width with padding if needed
        if current_width < target_width:
            # Need to pad horizontally
            pad_width = target_width - current_width
            paddings = [
                [0, 0],  # No vertical padding
                [pad_width // 2, pad_width - pad_width // 2],  # Horizontal padding
                [0, 0]   # No channel padding
            ]
            return tf.pad(image, paddings, mode='CONSTANT', constant_values=0)
        elif current_width > target_width:
            # Need to resize down to target width
            aspect_ratio = tf.cast(current_height, tf.float32) / tf.cast(current_width, tf.float32)
            new_height = tf.cast(tf.cast(target_width, tf.float32) * aspect_ratio, tf.int32)
            return tf.image.resize(image, [new_height, target_width])
        else:
            # Width already matches
            return image

In [14]:
#Enhanced Video Loading with Empty Frame Handling
def load_video(path: str, max_frames=75, target_width: int = 100, target_height: int = 50) -> np.ndarray:
    """
    Enhanced video loading with face detection and empty frame handling
    """
    cap = cv2.VideoCapture(path)
    frames = []
    current_crop = None
    frame_count = 0
    recalibrate_every = 30

    # Get total frames for better handling
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    if total_frames == 0:
        print(f"⚠️ Warning: No frames found in video: {path}")
        # Return padded empty frames
        empty_frames = [np.zeros((target_height, target_width)) for _ in range(max_frames)]
        return np.stack(empty_frames)

    while len(frames) < max_frames:
        ret, frame = cap.read()
        if not ret:
            break

        # Recalibrate crop region periodically
        if frame_count % recalibrate_every == 0:
            current_crop = measurements(frame)  # Use original BGR frame for landmark detection

        # Convert to grayscale first
        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        frame_gray = np.expand_dims(frame_gray, axis=-1)  # Add channel dimension
        frame_tensor = tf.convert_to_tensor(frame_gray, dtype=tf.float32)

        # Apply current crop
        y_start, y_end, x_start, x_end = current_crop
        
        # Ensure crop boundaries are valid
        y_start = max(0, min(y_start, frame.shape[0] - 1))
        y_end = max(y_start + 1, min(y_end, frame.shape[0]))
        x_start = max(0, min(x_start, frame.shape[1] - 1))
        x_end = max(x_start + 1, min(x_end, frame.shape[1]))
        
        cropped = frame_tensor[y_start:y_end, x_start:x_end, :]

        # Resize with horizontal padding
        resized = resize_with_horizontal_padding(cropped, target_width, target_height)
        
        # Convert back to numpy and remove channel dimension for consistency
        frame_processed = resized.numpy().squeeze()
        
        # Ensure frame has correct dimensions
        if frame_processed.shape != (target_height, target_width):
            frame_processed = cv2.resize(frame_processed, (target_width, target_height))
        
        frames.append(frame_processed / 255.0)  # Normalize to [0, 1]
        frame_count += 1

    cap.release()

    # Handle case where video has fewer frames than max_frames
    while len(frames) < max_frames:
        if len(frames) > 0:
            # Repeat the last frame
            frames.append(frames[-1].copy())
        else:
            # If no frames were processed, add empty frames
            frames.append(np.zeros((target_height, target_width)))

    return np.stack(frames)

# Creating Pipelines

In [15]:
#Alignment and Dataset Loading Functions
def load_alignment(path):
    """Load alignment data from file"""
    with open(path, 'r') as f:
        lines = f.readlines()
    words = [line.strip().split()[-1] for line in lines if len(line.strip().split()) >= 3]
    return ' '.join(words)

def load_dataset(video_root, alignment_root, maxlen=75):
    """Load dataset from all speakers with enhanced video loading"""
    X, y = [], []
    all_labels = set()
    failed_videos = []

    for speaker in os.listdir(video_root):
        speaker_path = os.path.join(video_root, speaker)
        if not os.path.isdir(speaker_path):
            continue

        print(f"Processing speaker: {speaker}")
        for filename in os.listdir(speaker_path):
            if not filename.endswith('.mp4'):
                continue

            video_path = os.path.join(speaker_path, filename)
            base = os.path.splitext(filename)[0]
            alignment_path = os.path.join(alignment_root, speaker, base + '.txt')

            if not os.path.exists(alignment_path):
                print(f" Missing alignment for: {video_path}")
                continue

            try:
                # Use enhanced video loading
                frames = load_video(video_path, max_frames=maxlen)
                label = load_alignment(alignment_path)

                # Check if frames are valid (not all zeros)
                if np.sum(frames) == 0:
                    print(f" Empty frames detected in: {video_path}")
                    failed_videos.append(video_path)
                    continue

                X.append(frames)
                y.append(label)
                all_labels.update(label.split())
                
            except Exception as e:
                print(f" Error processing {video_path}: {str(e)}")
                failed_videos.append(video_path)
                continue

    if failed_videos:
        print(f"\n Failed to process {len(failed_videos)} videos:")
        for failed in failed_videos[:5]:  # Show first 5 failed videos
            print(f"  - {failed}")
        if len(failed_videos) > 5:
            print(f"  ... and {len(failed_videos) - 5} more")

    X = np.array(X)
    label_encoder = LabelEncoder()
    label_encoder.fit(list(all_labels))

    y_encoded = [label_encoder.transform(label.split()) for label in y]
    y_padded = pad_sequences(y_encoded, maxlen=20, padding='post', value=-1)

    print(f"\n Successfully loaded {len(X)} videos")
    print(f" Dataset shape: {X.shape}")
    print(f" Number of unique labels: {len(all_labels)}")

    return X, y_padded, label_encoder

# Creating Model 

In [16]:
#Model Architecture and CTC Loss
def ctc_lambda_func(args):
    """CTC loss function"""
    y_pred, labels, input_length, label_length = args
    return K.ctc_batch_cost(labels, y_pred, input_length, label_length)

def build_model(timesteps=75, features=500, num_classes=50):
    """Build the lip reading model with CTC loss"""
    input_data = Input(name='input', shape=(timesteps, 50, 100, 1))
    x = TimeDistributed(tf.keras.layers.Flatten())(input_data)
    x = TimeDistributed(Dense(128, activation='relu'))(x)
    x = Bidirectional(LSTM(128, return_sequences=True))(x)
    x = Dense(num_classes + 1, activation='softmax', name='softmax')(x)

    labels = Input(name='labels', shape=[None], dtype='int32')
    input_lengths = Input(name='input_lengths', shape=[1], dtype='int32')
    label_lengths = Input(name='label_lengths', shape=[1], dtype='int32')

    loss_out = Lambda(ctc_lambda_func, output_shape=(1,), name='ctc')([x, labels, input_lengths, label_lengths])

    model = Model(inputs=[input_data, labels, input_lengths, label_lengths], outputs=loss_out)
    model.compile(loss={'ctc': lambda y_true, y_pred: y_pred}, optimizer=Adam(learning_rate=0.001))

    return model, input_data, x

# Data Encoding and Pipeline

In [17]:
#Load and Prepare Data
video_root = "data/recordings/data"
alignment_root = "data/alignments"

print("🔄 Loading dataset...")
X, y_padded, label_encoder = load_dataset(video_root, alignment_root)

# Add channel dimension and prepare data
X = X[..., np.newaxis]  # add channel dimension
timesteps = X.shape[1]
batch_size = X.shape[0]

input_lengths = np.full((batch_size, 1), timesteps)
label_lengths = np.array([[len([l for l in seq if l != -1])] for seq in y_padded])

print(f" Data preparation complete!")
print(f"   - Input shape: {X.shape}")
print(f"   - Labels shape: {y_padded.shape}")
print(f"   - Timesteps: {timesteps}")
print(f"   - Batch size: {batch_size}")
print(f"   - Number of classes: {len(label_encoder.classes_)}")

# Save label encoder for later use
with open('label_encoder.pkl', 'wb') as f:
    pickle.dump(label_encoder, f)
print("Label encoder saved to 'label_encoder.pkl'")

🔄 Loading dataset...
Processing speaker: s3


I0000 00:00:1754545958.758066  147450 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13759 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5


Processing speaker: s2

 Successfully loaded 22 videos
 Dataset shape: (22, 75, 50, 100)
 Number of unique labels: 36
 Data preparation complete!
   - Input shape: (22, 75, 50, 100, 1)
   - Labels shape: (22, 20)
   - Timesteps: 75
   - Batch size: 22
   - Number of classes: 36
Label encoder saved to 'label_encoder.pkl'


In [18]:
#Checkpoints
import shutil 
checkpoint_dir = 'checkpoints'
last_epoch_file = os.path.join(checkpoint_dir, 'last_epoch.txt')
latest_weights_path = os.path.join(checkpoint_dir, 'latest_checkpoint.weights.h5')
full_model_path = os.path.join(checkpoint_dir, 'latest_full_model.h5')

# Ask user if they want to start from scratch
# Build model
print("🔧 Building model...")
model, input_tensor, y_pred_tensor = build_model(timesteps=timesteps, num_classes=len(label_encoder.classes_))

user_input = input("🔁 Do you want to start training from scratch and delete old checkpoints? (y/n): ").strip().lower()

if user_input == 'y':
    if os.path.exists(checkpoint_dir):
        shutil.rmtree(checkpoint_dir)
        print("🗑️ Old checkpoints deleted.")
    os.makedirs(checkpoint_dir, exist_ok=True)
    initial_epoch = 0
    print("🆕 Starting training from scratch.")
else:
    os.makedirs(checkpoint_dir, exist_ok=True)
    initial_epoch = 0
    if os.path.exists(latest_weights_path):
        print(f"🧠 Loading weights from: {latest_weights_path}")
        model.load_weights(latest_weights_path)

        if os.path.exists(last_epoch_file):
            with open(last_epoch_file, 'r') as f:
                initial_epoch = int(f.read())
            print(f"⏩ Resuming from epoch {initial_epoch}")
        else:
            print("⚠️ Epoch tracking file missing. Starting from epoch 0.")
    else:
        print("🚀 No existing weights found. Starting training from scratch.")

# Build model
print("🔧 Building model...")
model, input_tensor, y_pred_tensor = build_model(timesteps=timesteps, num_classes=len(label_encoder.classes_))

# Set up checkpoint callback
def save_epoch_callback(epoch, logs=None):
    with open(last_epoch_file, 'w') as f:
        f.write(str(epoch + 1))  # Save next epoch index

checkpoint_latest = ModelCheckpoint(
    filepath=latest_weights_path,
    save_weights_only=True,
    save_freq='epoch'
)

callbacks = [checkpoint_latest, tf.keras.callbacks.LambdaCallback(on_epoch_end=save_epoch_callback)]


🔧 Building model...


🔁 Do you want to start training from scratch and delete old checkpoints? (y/n):  y


🗑️ Old checkpoints deleted.
🆕 Starting training from scratch.
🔧 Building model...


# Model Train and outputs

In [ ]:
#Train
print("Starting training...")
history = model.fit(
    x={
        'input': X,
        'labels': y_padded,
        'input_lengths': input_lengths,
        'label_lengths': label_lengths
    },
    y=np.zeros((batch_size,)),
    epochs=2000,
    initial_epoch=initial_epoch,
    callbacks=callbacks,
    verbose=1
)

print("✅ Training completed!")

# Save final model (overwrite last full)
model.save(full_model_path)
print(f"Full model saved to '{full_model_path}'")

#Save prediction model
predict_model = Model(inputs=input_tensor, outputs=y_pred_tensor)
#predict_model.save('lipreading_model_predict.h5')  # Optional
predict_model.save('predict_model.h5')
print("Prediction model saved to 'lipreading_model_predict.h5'")

#Save training history
with open('training_history.pkl', 'wb') as f:
    pickle.dump(history.history, f)
print("Training history saved to 'training_history.pkl'")


Starting training...
Epoch 1/2000


I0000 00:00:1754545985.464700  147537 cuda_dnn.cc:529] Loaded cuDNN version 90501


1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step - loss: 235.0572
Epoch 2/2000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 196ms/step - loss: 190.4828
Epoch 3/2000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 182ms/step - loss: 146.6686
Epoch 4/2000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 172ms/step - loss: 102.3679
Epoch 5/2000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 173ms/step - loss: 58.3198
Epoch 6/2000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 171ms/step - loss: 33.5008
Epoch 7/2000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 180ms/step - loss: 25.9266
Epoch 8/2000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 175ms/step - loss: 25.4466
Epoch 9/2000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 177ms/step - loss: 26.5430
Epoch 10/2000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 177ms/step - loss: 27.5243
Epoch 11/2000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 154ms/step - loss: 27.9272
Epoch 12/2000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 175ms/step - loss: 27.3459
Epoch 13/2000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 173ms/step - loss: 26.8094
Epoch 14/2000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 174ms/step - loss: 26.1903
Epoch 15/2000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 155ms/step - 

In [ ]:
#Testing and Inference Functions
def load_trained_model(model_path='lipreading_model_predict.h5', label_encoder_path='label_encoder.pkl'):
    """Load trained model and label encoder"""
    try:
        model = load_model(model_path)
        with open(label_encoder_path, 'rb') as f:
            label_encoder = pickle.load(f)
        print(f"✅ Model loaded from {model_path}")
        print(f"✅ Label encoder loaded from {label_encoder_path}")
        return model, label_encoder
    except Exception as e:
        print(f"❌ Error loading model: {str(e)}")
        return None, None

def predict_video(video_path, model=None, label_encoder=None, max_frames=75):
    """Predict text from a video file"""
    if model is None or label_encoder is None:
        model, label_encoder = load_trained_model()
        if model is None:
            return "Error: Could not load model"
    
    try:
        # Load and preprocess video
        print(f"🔄 Processing video: {video_path}")
        frames = load_video(video_path, max_frames=max_frames)
        
        # Add batch and channel dimensions
        frames = frames[np.newaxis, ..., np.newaxis]
        
        # Predict
        y_pred = model.predict(frames, verbose=0)
        
        # Decode prediction
        decoded, _ = tf.keras.backend.ctc_decode(
            y_pred,
            input_length=np.ones(y_pred.shape[0]) * y_pred.shape[1],
            greedy=True
        )
        
        # Convert to words
        sequence = decoded[0].numpy()[0]
        words = [label_encoder.inverse_transform([i])[0] for i in sequence if i != -1]
        predicted_text = ' '.join(words)
        
        print(f"🔤 Predicted: {predicted_text}")
        return predicted_text
        
    except Exception as e:
        error_msg = f"❌ Error predicting video {video_path}: {str(e)}"
        print(error_msg)
        return error_msg

def test_on_training_data(model=None, label_encoder=None, num_samples=5):
    """Test model on training data samples"""
    if model is None or label_encoder is None:
        model, label_encoder = load_trained_model()
        if model is None:
            return
    
    print(f" Testing on {min(num_samples, len(X))} training samples...")
    
    for i in range(min(num_samples, len(X))):
        # Get single sample
        sample = X[i:i+1]
        
        # Predict
        y_pred = model.predict(sample, verbose=0)
        
        # Decode prediction
        decoded, _ = tf.keras.backend.ctc_decode(
            y_pred,
            input_length=np.ones(y_pred.shape[0]) * y_pred.shape[1],
            greedy=True
        )
        
        # Convert to words
        sequence = decoded[0].numpy()[0]
        predicted_words = [label_encoder.inverse_transform([j])[0] for j in sequence if j != -1]
        predicted_text = ' '.join(predicted_words)
        
        # Get ground truth
        true_sequence = y_padded[i]
        true_words = [label_encoder.inverse_transform([j])[0] for j in true_sequence if j != -1]
        true_text = ' '.join(true_words)
        
        print(f"\n Sample {i+1}:")
        print(f"   True:      {true_text}")
        print(f"   Predicted: {predicted_text}")

# Test on training data
test_on_training_data(predict_model, label_encoder)

# Model Testing and Evaulation

In [ ]:
#Test on New Videos
def test_new_video(video_path):
    """Quick function to test a new video"""
    return predict_video(video_path, predict_model, label_encoder)

# Example usage:
# Uncomment and modify the path below to test on a new video
test_video_path = "data/recordings/data/s3/dnma1d.mp4"
result = test_new_video(test_video_path)
print(f"Result: {result}")

print("🎬 Ready to test new videos!")
print("Usage: test_new_video('path/to/your/video.mp4')")
print("\n📁 Available models:")
print("   - lipreading_model_full.h5 (full training model)")
print("   - lipreading_model_predict.h5 (inference model)")
print("   - label_encoder.pkl (label encoder)")
print("   - training_history.pkl (training history)")

# Function to batch test multiple videos
def batch_test_videos(video_paths):
    """Test multiple videos at once"""
    results = {}
    for path in video_paths:
        if os.path.exists(path):
            results[path] = predict_video(path, predict_model, label_encoder)
        else:
            results[path] = f"❌ File not found: {path}"
    return results

print("\n🔧 Additional functions available:")
print("   - batch_test_videos(['path1.mp4', 'path2.mp4', ...]) - Test multiple videos")
print("   - load_trained_model() - Load saved model and encoder")
print("   - test_on_training_data(num_samples=5) - Test on training data")

In [12]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 3939318106915917679
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 14428209152
locality {
  bus_id: 1
  links {
  }
}
incarnation: 16577857435198946522
physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
xla_global_id: 416903419
]


I0000 00:00:1754545935.038347  147450 gpu_device.cc:2019] Created device /device:GPU:0 with 13759 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5


In [2]:
import gradio as gr
import tensorflow as tf
import numpy as np
import cv2
import imageio
import os
import tempfile
import pickle
from pathlib import Path
import logging

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class LipReadingPredictor:
    def __init__(self, model_path="predict_model.h5", encoder_path="label_encoder.pkl"):
        self.model = None
        self.label_encoder = None
        self.load_model(model_path, encoder_path)
        
    def load_model(self, model_path, encoder_path):
        try:
            if os.path.exists(model_path):
                self.model = tf.keras.models.load_model(model_path)
                logger.info(f"Model loaded successfully from {model_path}")
                logger.info(f"Model input shape: {self.model.input_shape}")
            else:
                logger.error(f"Model file not found: {model_path}")
                
            if os.path.exists(encoder_path):
                with open(encoder_path, "rb") as f:
                    self.label_encoder = pickle.load(f)
                logger.info(f"Label encoder loaded. Classes: {len(self.label_encoder.classes_)}")
                logger.info(f"Available classes: {list(self.label_encoder.classes_)}")
            else:
                logger.error(f"Label encoder file not found: {encoder_path}")
                
        except Exception as e:
            logger.error(f"Error loading model/encoder: {str(e)}")
    
    def preprocess_video(self, video_path, max_frames=75, target_size=(100, 50)):
        """Preprocess video for lip reading prediction"""
        try:
            cap = cv2.VideoCapture(video_path)
            if not cap.isOpened():
                raise ValueError(f"Could not open video file: {video_path}")
            
            frames = []
            frame_count = 0
            
            while True:
                ret, frame = cap.read()
                if not ret:
                    break
                    
                # Convert to grayscale
                if len(frame.shape) == 3:
                    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                else:
                    gray = frame
                
                # Resize to target size
                resized = cv2.resize(gray, target_size)  # (width, height)
                frames.append(resized)
                frame_count += 1
            
            cap.release()
            
            if len(frames) == 0:
                raise ValueError("No frames extracted from video")
            
            logger.info(f"Extracted {len(frames)} frames from video")
            
            # Pad or trim to max_frames
            if len(frames) > max_frames:
                frames = frames[:max_frames]
            elif len(frames) < max_frames:
                # Pad with zeros or repeat last frame
                last_frame = frames[-1] if frames else np.zeros(target_size[::-1])  # (height, width)
                padding_needed = max_frames - len(frames)
                frames.extend([last_frame] * padding_needed)
            
            # Convert to numpy array and normalize
            frames_array = np.array(frames, dtype=np.float32)
            frames_array = frames_array / 255.0  # Normalize to [0, 1]
            
            # Reshape for model input: (batch_size, frames, height, width, channels)
            frames_input = frames_array.reshape(1, max_frames, target_size[1], target_size[0], 1)
            
            logger.info(f"Preprocessed video shape: {frames_input.shape}")
            return frames_input, frames
            
        except Exception as e:
            logger.error(f"Error preprocessing video: {str(e)}")
            raise
    
    def create_preview_gif(self, frames, output_path=None):
        """Create a GIF preview from frames"""
        try:
            if output_path is None:
                output_path = os.path.join(tempfile.gettempdir(), f"preview_{os.getpid()}.gif")
            
            # Convert frames to uint8 for GIF creation
            frames_uint8 = [(frame * 255).astype(np.uint8) if frame.max() <= 1 else frame.astype(np.uint8) 
                           for frame in frames[:30]]  # Limit to first 30 frames for GIF
            
            imageio.mimsave(output_path, frames_uint8, fps=10, loop=0)
            return output_path
        except Exception as e:
            logger.error(f"Error creating GIF: {str(e)}")
            return None
    
    def decode_prediction_safe(self, y_pred):
        """Safely decode CTC prediction with proper error handling"""
        try:
            # Method 1: Try CTC decode (for CTC-trained models)
            try:
                decoded, _ = tf.keras.backend.ctc_decode(
                    y_pred,
                    input_length=np.ones(y_pred.shape[0]) * y_pred.shape[1],
                    greedy=True
                )
                
                # Convert to numpy and get sequence
                sequence = decoded[0].numpy()[0]
                
                # Filter out blank tokens (-1) and invalid indices
                valid_indices = [i for i in sequence if i >= 0 and i < len(self.label_encoder.classes_)]
                
                if valid_indices:
                    words = []
                    for idx in valid_indices:
                        try:
                            word = self.label_encoder.classes_[idx]
                            words.append(word)
                        except IndexError:
                            logger.warning(f"Index {idx} out of range for label encoder")
                            continue
                    
                    predicted_text = ' '.join(words) if words else "No valid prediction"
                    confidence = np.max(y_pred)
                    return predicted_text, confidence
                    
            except Exception as ctc_error:
                logger.warning(f"CTC decode failed: {ctc_error}")
                
            # Method 2: Fallback to simple argmax (for non-CTC models)
            try:
                if len(y_pred.shape) == 3:  # (batch, timesteps, classes)
                    # Take argmax over time steps
                    predicted_indices = np.argmax(y_pred[0], axis=-1)
                elif len(y_pred.shape) == 2:  # (batch, classes)
                    predicted_indices = [np.argmax(y_pred[0])]
                else:
                    raise ValueError(f"Unexpected prediction shape: {y_pred.shape}")
                
                # Filter valid indices and decode
                words = []
                for idx in predicted_indices:
                    if 0 <= idx < len(self.label_encoder.classes_):
                        word = self.label_encoder.classes_[idx]
                        words.append(word)
                
                # Remove duplicates while preserving order
                unique_words = []
                for word in words:
                    if word not in unique_words:
                        unique_words.append(word)
                
                predicted_text = ' '.join(unique_words) if unique_words else "No valid prediction"
                confidence = np.max(y_pred)
                return predicted_text, confidence
                
            except Exception as argmax_error:
                logger.error(f"Argmax decode failed: {argmax_error}")
                
            # Method 3: Final fallback - return most probable class
            try:
                max_prob_idx = np.argmax(y_pred.flatten())
                if max_prob_idx < len(self.label_encoder.classes_):
                    predicted_text = self.label_encoder.classes_[max_prob_idx]
                    confidence = np.max(y_pred)
                    return predicted_text, confidence
                else:
                    return "Unknown", np.max(y_pred)
                    
            except Exception as final_error:
                logger.error(f"Final fallback failed: {final_error}")
                return "Error in prediction", 0.0
                
        except Exception as e:
            logger.error(f"All decode methods failed: {str(e)}")
            return "Decode error", 0.0
    
    def predict(self, video_path):
        """Main prediction function with safe decoding"""
        if self.model is None:
            return "Model not loaded", None, "❌ Error: Model not loaded"
        
        if self.label_encoder is None:
            return "Label encoder not loaded", None, "❌ Error: Label encoder not loaded"
        
        try:
            # Preprocess video
            frames_input, frames_list = self.preprocess_video(video_path)
            
            # Make prediction
            logger.info("Making prediction...")
            predictions = self.model.predict(frames_input, verbose=0)
            
            logger.info(f"Raw prediction shape: {predictions.shape}")
            
            # Safely decode prediction
            predicted_text, confidence = self.decode_prediction_safe(predictions)
            
            logger.info(f"Decoded prediction: {predicted_text}, Confidence: {confidence:.4f}")
            
            # Create preview GIF
            gif_path = self.create_preview_gif(frames_list)
            
            # Prepare detailed output
            status_msg = f"✅ Prediction: {predicted_text} (Confidence: {confidence:.2%})"
            detailed_text = f"{predicted_text} (Confidence: {confidence:.2%})"
            
            logger.info(f"Prediction complete: {predicted_text}")
            return detailed_text, gif_path, status_msg
            
        except Exception as e:
            error_msg = f"❌ Error during prediction: {str(e)}"
            logger.error(error_msg)
            return "Error", None, error_msg

# Standalone inference functions
def load_trained_model(model_path='lipreading_model_predict.h5', label_encoder_path='label_encoder.pkl'):
    """Load trained model and label encoder"""
    try:
        from tensorflow.keras.models import load_model
        model = load_model(model_path)
        with open(label_encoder_path, 'rb') as f:
            label_encoder = pickle.load(f)
        print(f"✅ Model loaded from {model_path}")
        print(f"✅ Label encoder loaded from {label_encoder_path}")
        return model, label_encoder
    except Exception as e:
        print(f"❌ Error loading model: {str(e)}")
        return None, None

def load_video(video_path, max_frames=75, target_size=(100, 50)):
    """Load and preprocess video (standalone function)"""
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise ValueError(f"Could not open video file: {video_path}")
    
    frames = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break
            
        # Convert to grayscale
        if len(frame.shape) == 3:
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        else:
            gray = frame
        
        # Resize to target size
        resized = cv2.resize(gray, target_size)
        frames.append(resized)
    
    cap.release()
    
    if len(frames) == 0:
        raise ValueError("No frames extracted from video")
    
    # Pad or trim to max_frames
    if len(frames) > max_frames:
        frames = frames[:max_frames]
    elif len(frames) < max_frames:
        last_frame = frames[-1] if frames else np.zeros(target_size[::-1])
        padding_needed = max_frames - len(frames)
        frames.extend([last_frame] * padding_needed)
    
    # Convert to numpy array and normalize
    frames_array = np.array(frames, dtype=np.float32)
    frames_array = frames_array / 255.0
    
    return frames_array

def predict_video_safe(video_path, model=None, label_encoder=None, max_frames=75):
    """Predict text from a video file with safe decoding"""
    if model is None or label_encoder is None:
        model, label_encoder = load_trained_model()
        if model is None:
            return "Error: Could not load model"
    
    try:
        # Load and preprocess video
        print(f"🔄 Processing video: {video_path}")
        frames = load_video(video_path, max_frames=max_frames)
        
        # Add batch and channel dimensions
        frames = frames[np.newaxis, ..., np.newaxis]
        
        # Predict
        y_pred = model.predict(frames, verbose=0)
        print(f"Prediction shape: {y_pred.shape}")
        
        # Safe decoding
        try:
            # Try CTC decode first
            decoded, _ = tf.keras.backend.ctc_decode(
                y_pred,
                input_length=np.ones(y_pred.shape[0]) * y_pred.shape[1],
                greedy=True
            )
            
            sequence = decoded[0].numpy()[0]
            valid_indices = [i for i in sequence if i >= 0 and i < len(label_encoder.classes_)]
            
            if valid_indices:
                words = [label_encoder.classes_[i] for i in valid_indices]
                predicted_text = ' '.join(words)
            else:
                predicted_text = "No valid prediction"
                
        except Exception as ctc_error:
            print(f"CTC decode failed, using argmax: {ctc_error}")
            # Fallback to argmax
            if len(y_pred.shape) == 3:
                predicted_indices = np.argmax(y_pred[0], axis=-1)
            else:
                predicted_indices = [np.argmax(y_pred[0])]
            
            words = []
            for idx in predicted_indices:
                if 0 <= idx < len(label_encoder.classes_):
                    words.append(label_encoder.classes_[idx])
            
            # Remove duplicates while preserving order
            unique_words = []
            for word in words:
                if word not in unique_words:
                    unique_words.append(word)
            
            predicted_text = ' '.join(unique_words) if unique_words else "No valid prediction"
        
        print(f"🔤 Predicted: {predicted_text}")
        return predicted_text
        
    except Exception as e:
        error_msg = f"❌ Error predicting video {video_path}: {str(e)}"
        print(error_msg)
        return error_msg

def test_on_training_data_safe(X, y_padded, model=None, label_encoder=None, num_samples=5):
    """Test model on training data samples with safe decoding"""
    if model is None or label_encoder is None:
        model, label_encoder = load_trained_model()
        if model is None:
            return
    
    print(f"🧪 Testing on {min(num_samples, len(X))} training samples...")
    
    for i in range(min(num_samples, len(X))):
        try:
            # Get single sample
            sample = X[i:i+1]
            
            # Predict
            y_pred = model.predict(sample, verbose=0)
            
            # Safe decode prediction
            try:
                decoded, _ = tf.keras.backend.ctc_decode(
                    y_pred,
                    input_length=np.ones(y_pred.shape[0]) * y_pred.shape[1],
                    greedy=True
                )
                
                sequence = decoded[0].numpy()[0]
                valid_indices = [j for j in sequence if j >= 0 and j < len(label_encoder.classes_)]
                predicted_words = [label_encoder.classes_[j] for j in valid_indices]
                predicted_text = ' '.join(predicted_words) if predicted_words else "No valid prediction"
                
            except Exception:
                # Fallback decoding
                if len(y_pred.shape) == 3:
                    predicted_indices = np.argmax(y_pred[0], axis=-1)
                else:
                    predicted_indices = [np.argmax(y_pred[0])]
                
                predicted_words = []
                for idx in predicted_indices:
                    if 0 <= idx < len(label_encoder.classes_):
                        predicted_words.append(label_encoder.classes_[idx])
                
                predicted_text = ' '.join(predicted_words) if predicted_words else "No valid prediction"
            
            # Get ground truth (safe)
            true_sequence = y_padded[i]
            valid_true_indices = [j for j in true_sequence if j >= 0 and j < len(label_encoder.classes_)]
            true_words = [label_encoder.classes_[j] for j in valid_true_indices]
            true_text = ' '.join(true_words) if true_words else "No valid ground truth"
            
            print(f"\n📊 Sample {i+1}:")
            print(f"   True:      {true_text}")
            print(f"   Predicted: {predicted_text}")
            
        except Exception as e:
            print(f"❌ Error processing sample {i+1}: {str(e)}")
            continue

# Initialize predictor
predictor = LipReadingPredictor()

# Sample directory setup
sample_dir = "data/recordings/data/s2"
if os.path.exists(sample_dir):
    sample_files = [f for f in os.listdir(sample_dir) if f.lower().endswith(('.mp4', '.avi', '.mov', '.mkv'))]
else:
    sample_files = []
    logger.warning(f"Sample directory not found: {sample_dir}")

def predict_from_upload(video_file):
    """Handle uploaded video prediction"""
    if video_file is None:
        return "No video uploaded", None, "❌ Please upload a video file"
    
    return predictor.predict(video_file)

def predict_from_sample(sample_name):
    """Handle sample video prediction"""
    if not sample_name:
        return "No sample selected", None, "❌ Please select a sample video"
    
    sample_path = os.path.join(sample_dir, sample_name)
    if not os.path.exists(sample_path):
        return "Sample not found", None, f"❌ Sample file not found: {sample_name}"
    
    return predictor.predict(sample_path)

def get_model_info():
    """Get model information for display"""
    info = []
    if predictor.model:
        info.append(f"✅ Model loaded: {predictor.model.input_shape}")
    else:
        info.append("❌ Model not loaded")
    
    if predictor.label_encoder:
        info.append(f"✅ Label encoder loaded: {len(predictor.label_encoder.classes_)} classes")
        info.append(f"Classes: {', '.join(predictor.label_encoder.classes_[:10])}{'...' if len(predictor.label_encoder.classes_) > 10 else ''}")
    else:
        info.append("❌ Label encoder not loaded")
    
    return "\n".join(info)

# Create Gradio interface
with gr.Blocks(title="LumaVoice Lip Reading", theme=gr.themes.Soft()) as demo:
    gr.Markdown("""
    # 🎤 LumaVoice: Hindi Lip Reading System
    
    Upload a silent video or select a sample to predict what words are being spoken.
    """)
    
    # Model status
    with gr.Row():
        model_status = gr.Textbox(
            label="🔧 System Status", 
            value=get_model_info(),
            lines=4,
            interactive=False
        )
    
    with gr.Row():
        # Left column - Input
        with gr.Column(scale=1):
            gr.Markdown("### 📹 Input Video")
            
            video_input = gr.Video(
                label="Upload Video File",
                sources=["upload"],
                height=300
            )
            
            upload_btn = gr.Button(
                "🔍 Analyze Uploaded Video", 
                variant="primary",
                size="lg"
            )
            
            gr.Markdown("### 📁 Or Try Sample Videos")
            
            if sample_files:
                sample_dropdown = gr.Dropdown(
                    choices=sample_files,
                    label="Select Sample Video",
                    value=sample_files[0] if sample_files else None
                )
                
                sample_btn = gr.Button(
                    "🔍 Analyze Sample Video",
                    variant="secondary",
                    size="lg"
                )
            else:
                gr.Markdown("*No sample videos found*")
        
        # Right column - Output
        with gr.Column(scale=1):
            gr.Markdown("### 📝 Prediction Results")
            
            predicted_text = gr.Textbox(
                label="Predicted Text",
                placeholder="Prediction will appear here...",
                lines=2,
                interactive=False
            )
            
            confidence_info = gr.Textbox(
                label="Status",
                placeholder="Status will appear here...",
                lines=2,
                interactive=False
            )
            
            gr.Markdown("### 🎬 Video Preview")
            
            preview_gif = gr.Image(
                label="Processed Frames",
                height=200,
                type="filepath"
            )
    
    # Event handlers
    upload_btn.click(
        fn=predict_from_upload,
        inputs=[video_input],
        outputs=[predicted_text, preview_gif, confidence_info]
    )
    
    if sample_files:
        sample_btn.click(
            fn=predict_from_sample,
            inputs=[sample_dropdown],
            outputs=[predicted_text, preview_gif, confidence_info]
        )

if __name__ == "__main__":
    # Example usage of standalone functions:
    # model, label_encoder = load_trained_model()
    # if model and label_encoder:
    #     result = predict_video_safe("path/to/video.mp4", model, label_encoder)
    #     print(f"Prediction: {result}")
    
    # Try multiple ports or let Gradio find an available one
    demo.launch(share=True)

I0000 00:00:1753183711.498592   46436 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13759 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
INFO:__main__:Model loaded successfully from predict_model.h5
INFO:__main__:Model input shape: (None, 75, 50, 100, 1)
INFO:__main__:Label encoder loaded. Classes: 36
INFO:__main__:Available classes: [np.str_('A'), np.str_('B'), np.str_('E'), np.str_('F'), np.str_('G'), np.str_('H'), np.str_('aath'), np.str_('ab'), np.str_('bata'), np.str_('chaar'), np.str_('chhah'), np.str_('dheere'), np.str_('dikhao'), np.str_('do'), np.str_('ek'), np.str_('hamesha'), np.str_('hara'), np.str_('jaldi'), np.str_('kaala'), np.str_('kal'), np.str_('ke'), np.str_('laal'), np.str_('mein'), np.str_('nau'), np.str_('neeche'), np.str_('neela'), np.str_('oopar'), np.str_('paanch'), np.str_('paas'), np.str_('par'), np.str_('peela'), np.str_('saat'), np.str_('safed'), np.str_('set'), np.str_('sho

* Running on local URL:  http://127.0.0.1:7860


INFO:httpx:HTTP Request: GET https://api.gradio.app/pkg-version "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://api.gradio.app/v3/tunnel-request "HTTP/1.1 200 OK"


* Running on public URL: https://74968ae376116bb009.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


INFO:httpx:HTTP Request: HEAD https://74968ae376116bb009.gradio.live "HTTP/1.1 200 OK"
